# DomoAuth

> Fill in a module description here


In [ ]:
# | default_exp DomoAuth

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
# | exporti

from dataclasses import dataclass, field

import aiohttp
from typing import Optional, Union
import nbdev_domo.ResponseGetData as rgd
from nbdev_domo.Transport import TransportAsync

# Authentication Routes

The nbdev_domo library was designed to support object-oriented / class-based programming; however, all routes are stand-alone functions for users who prefer a functional programming approach

## Full Auth Route - username and password authentication

If SSO is enabled, users must have direct-sign-on ([SSO KB](https://domo-support.domo.com/s/article/360042934374?language=en_US)) access to use this authentication method.


In [ ]:
# | export
async def get_full_auth(
    domo_instance: str,  # domo_instance.domo.com
    domo_username: str,  # email address
    domo_password: str,
    session: Optional[aiohttp.ClientSession] = None,
) -> rgd.ResponseGetData:
    """uses username and password authentication to retrieve a full_auth access token"""

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password,
    }

    transport = TransportAsync(session=session)

    return await transport.post(url=url, body=body, session=session)

#### Sample Implementations of get_full_auth


In [ ]:
domo_instance = "domo-dojo"
domo_username = "test12@domo.com"
domo_password = "test1234"

res = await get_full_auth(domo_instance, domo_username, domo_password)

# The 200 response confirms we were able to get a response from the credentials API,
# however success was FALSE because we sent invalid credentials
test_eq(res.status, 200)
test_eq(res.response.get("success"), False)

res

ResponseGetData(status=200, response={'success': False, 'reason': 'INVALID_CREDENTIALS'}, is_success=True, auth_header=None)

In [ ]:
domo_instance = "test"
res = await get_full_auth(domo_instance, domo_username, domo_password)

# the 403 response confirms that there is no domo instance 'test.domo.com'
test_eq(res.status, 403)
res

ResponseGetData(status=403, response='Forbidden', is_success=False, auth_header=None)

## Access Token Auth Route - access_token authentication

This access_token based authentication assumes the user has been provided a valid access token from Domo > Admin > Authentication > Access Token so authentication routes are actually not required.

Per the Domo JavaCLI implementation, users can test the validity of the access_token agains the 'me' API

!! Note about the Me API !!
It appears that access_token authentication will always authenticate against the 'correct' domo_instance regardless of the parameter provided. Ths appears to be a bug in the domo_api


In [ ]:
# | export
async def test_access_token(
    domo_access_token: str,  # as provided in Domo > Admin > Authentication > AccessTokens
    domo_instance: str,  # <domo_instance>.domo.com
    session: Optional[aiohttp.ClientSession] = None,
):
    """
    will attempt to validate against the 'me' API.
    This is the same authentication test the Domo Java CLI uses.
    """

    url = f"https://{domo_instance}.domo.com/api/content/v2/users/me"

    transport = TransportAsync(
        session=session, auth_header={"X-DOMO-Developer-Token": domo_access_token}
    )

    return await transport.get(url=url)

#### Sample implementation of `test_access_token`


In [ ]:
domo_instance = "test"
domo_access_token = "a9c9c837775a981121fc57b4c78550d28c8784b6b0f4c69c"

await test_access_token(
    domo_instance=domo_instance, domo_access_token=domo_access_token
)

ResponseGetData(status=200, response={'id': 1893952720, 'invitorUserId': 587894148, 'displayName': 'Jae Wilson', 'userName': 'jae@onyxreporting.com', 'emailAddress': 'jae@onyxreporting.com', 'avatarKey': 'c605f478-0cd2-4451-9fd4-d82090b71e66', 'accepted': True, 'userType': 'USER', 'modified': 1651692505000, 'created': 1588960518, 'role': 'Admin', 'rights': 63, 'active': True, 'pending': False, 'systemUser': False, 'anonymous': False}, is_success=True, auth_header={'X-DOMO-Developer-Token': 'a9c9c837775a981121fc57b4c78550d28c8784b6b0f4c69c'})

## Developer Auth Route - client_id and secret authentication


In [ ]:
# | export
async def get_developer_auth(
    domo_client_id: str,
    domo_client_secret: str,
    session: Optional[aiohttp.ClientSession] = None,
) -> rgd.ResponseGetData:

    """
    only use for authenticating against apis documented under developer.domo.com
    """

    url = "https://api.domo.com/oauth/token?grant_type=client_credentials"

    is_close_session = False

    if not session:
        is_close_session = True
        session = aiohttp.ClientSession(
            auth=aiohttp.BasicAuth(domo_client_id, domo_client_secret)
        )

    transport = TransportAsync(session=session)
    res = await transport.get(url=url)

    if is_close_session:
        await session.close()

    return res

#### Sample Implementations of get_developer_auth

The 401 response is expected because we are using invalid credentials


In [ ]:
domo_client_id = "test_id"
domo_client_secret = "test_secret"

await get_developer_auth(domo_client_id, domo_client_secret)

ResponseGetData(status=401, response='Unauthorized', is_success=False, auth_header=None)

# Authentication Classes


In [ ]:
# | hide
# different auth flows will have different required and optional parameters.
# The libraries use inheritance to mix-in parameters and methods as appropriate.
# The DomoAuth class is an abstract class with stubbed abstract methods which should be overwritten in the actual implementation.

In [ ]:
# | exporti
@dataclass
class _DomoAuth_Required:
    """required parameters for all Domo Auth classes"""

    domo_instance: str

    def __post_init__(self):
        if self.domo_instance:
            self.set_manual_login()

    def set_manual_login(self):
        self.url_manual_login = (
            f"https://{self.domo_instance}.domo.com/auth/index?domoManualLogin=true"
        )


@dataclass
class _DomoAuth_Optional:
    """optional parameters for Domo Auth classes that should be defined during initialization or during the Token / Authenitcation process"""

    token: Optional[str] = field(default=None, repr=False)
    token_name: Optional[str] = field(default=None)
    user_id: Optional[str] = field(default=None, repr=False)
    auth_header: dict = field(default_factory=dict, repr=False)

    url_manual_login: Optional[str] = None

    async def get_auth_token(self) -> Union[str, None]:
        """placeholder method"""
        pass

    async def generate_auth_header(self) -> Union[dict, None]:
        """returns auth header appropriate for this authentication method"""
        pass

In [ ]:
# | export
@dataclass
class DomoAuth(_DomoAuth_Optional, _DomoAuth_Required):
    """abstract DomoAuth class with all parameters and methods common across all Auth methods"""

    pass


In [ ]:
# Attributes of `DomoAuth`
domo_instance = "test"
[attr for attr in dir(DomoAuth(domo_instance)) if not attr.startswith("__")]

['auth_header',
 'domo_instance',
 'generate_auth_header',
 'get_auth_token',
 'set_manual_login',
 'token',
 'token_name',
 'url_manual_login',
 'user_id']

In [ ]:
# | hide
# validate can print manual login link
domo_instance = "test"

da = DomoAuth(domo_instance)
da.url_manual_login

'https://test.domo.com/auth/index?domoManualLogin=true'

## DomoAuth Error Classes

Used to raise exceptions during authentication process.


In [ ]:
# | exporti
class DomoErrror(Exception):
    """base exception"""

    def __init__(
        self,
        status: Optional[int] = None,  # API request status
        message: str = "error",  # <domo_instance>.domo.com
        domo_instance: Optional[str] = None,
    ):

        instance_str = f" at {domo_instance}" if domo_instance else ""
        status_str = f"Status {status} - " if status else ""
        self.message = f"{status_str}{message}{instance_str}"
        super().__init__(self.message)

In [ ]:
# | export
class InvalidCredentialsError(DomoErrror):
    """return invalid credentials sent to API"""

    def __init__(
        self,
        status: Optional[int] = None,  # API request status
        message="invalid credentials",
        domo_instance: Optional[str] = None,
    ):

        super().__init__(status=status, message=message, domo_instance=domo_instance)


class InvalidInstanceError(DomoErrror):
    """return if invalid domo_instance sent to API"""

    def __init__(
        self,
        status: Optional[int] = None,
        message="invalid instance",
        domo_instance: Optional[str] = None,
    ):
        super().__init__(status=status, message=message, domo_instance=domo_instance)

In [ ]:
# | hide
# validates that Errors can compile
test_eq(isinstance(InvalidCredentialsError(), InvalidCredentialsError), True)
test_eq(isinstance(InvalidCredentialsError(), InvalidCredentialsError), True)
test_eq(isinstance(InvalidInstanceError(), InvalidInstanceError), True)

## DomoFullAuth


In [ ]:
# | exporti
@dataclass
class _DomoFullAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_username: str
    domo_password: str = field(repr=False)


In [ ]:
# | export
@dataclass
class DomoFullAuth(_DomoAuth_Optional, _DomoFullAuth_Required):
    """use for full authentication token"""

    async def generate_auth_header(self, token: str = None) -> dict:
        if not self.token:
            await self.get_auth_token()

        self.auth_header = {"x-domo-authentication": token or self.token}
        return self.auth_header

    async def get_auth_token(
        self,
        session: Optional[aiohttp.ClientSession] = None,
    ) -> str:
        """returns `token` if valid credentials provided else raises Exception and returns None"""

        res = await get_full_auth(
            domo_instance=self.domo_instance,
            domo_username=self.domo_username,
            domo_password=self.domo_password,
            session=session,
        )

        if res.is_success and res.response.get("reason") == "INVALID_CREDENTIALS":
            raise InvalidCredentialsError(
                status=res.status,
                message=str(res.response.get("reason")),
                domo_instance=self.domo_instance,
            )

        if res.status == 403:
            raise InvalidInstanceError(
                status=res.status,
                message="INVALID INSTANCE",
                domo_instance=self.domo_instance,
            )

        token = str(res.response.get("sessionToken"))
        self.token = token
        self.user_id = str(res.response.get("userId"))

        if not self.token_name:
            self.token_name = "full_auth"

        return self.token

#### sample implementations of DomoFullAuth


In [ ]:
domo_instance = "domo-dojo"
domo_username = "test12@domo.com"
domo_password = "test1234"

try:
    full_auth = DomoFullAuth(domo_instance, domo_username, domo_password)
    res = await full_auth.get_auth_token()

except InvalidCredentialsError as e:
    print(e)

Status 200 - INVALID_CREDENTIALS at domo-dojo


In [ ]:
domo_instance = "test"

try:
    full_auth = DomoFullAuth(domo_instance, domo_username, domo_password)
    await full_auth.get_auth_token()
except InvalidInstanceError as e:
    print(e)

Status 403 - INVALID INSTANCE at test


In [ ]:
# | hide
test_eq(isinstance(DomoAuth(domo_instance=domo_instance), DomoAuth), True)

test_eq(
    isinstance(
        DomoFullAuth(
            domo_instance=domo_instance,
            domo_password=domo_password,
            domo_username=domo_username,
        ),
        DomoFullAuth,
    ),
    True,
)

## DomoTokenAuth


In [ ]:
# | exporti
@dataclass
class _DomoTokenAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_access_token: str = field(repr=False)

In [ ]:
# | export
@dataclass
class DomoTokenAuth(_DomoAuth_Optional, _DomoTokenAuth_Required):
    """
    use for access_token authentication.
    Tokens are generated in domo > admin > access token
    Necessary in cases where direct sign on is not permitted
    """

    def generate_auth_header(self, token: str) -> dict:
        self.auth_header = {"x-domo-developer-token": token}
        return self.auth_header

    async def get_auth_token(
        self, session: Optional[aiohttp.ClientSession] = None
    ) -> str:
        """
        updates internal attributes
        having an access_token assumes pre-authenticaiton
        """

        res = await test_access_token(
            domo_instance=self.domo_instance,
            domo_access_token=self.domo_access_token,
            session=session,
        )

        if res.status == 401 and res.response == "Unauthorized":
            raise InvalidCredentialsError(
                status=res.status,
                message=res.response,
                domo_instance=self.domo_instance,
            )

        self.token = self.domo_access_token
        self.user_id = res.response.get("id")

        self.auth_header = self.generate_auth_header(token=self.token)

        if not self.token_name:
            self.token_name = "token_auth"

        return self.token

#### Sample implementation of DomoTokenAuth


In [ ]:
domo_instance = "test"
domo_access_token = "test_access_token"

domo_access_token = "a9c9c837775a981121fc57b4c78550d28c8784b6b0f4c69c"
try:
    domo_auth = DomoTokenAuth(domo_instance, domo_access_token)
    token = await domo_auth.get_auth_token()
    print(domo_auth.token)

except InvalidCredentialsError as e:
    print(e)

a9c9c837775a981121fc57b4c78550d28c8784b6b0f4c69c


## DomoDeveloperAuth


In [ ]:
# | exporti
@dataclass
class _DomoDeveloperAuth_Required(_DomoAuth_Required):
    """mix requied parameters for DomoFullAuth"""

    domo_client_id: str
    domo_client_secret: str = field(repr=False)

In [ ]:
# | export
@dataclass(init=False)
class DomoDeveloperAuth(_DomoAuth_Optional, _DomoDeveloperAuth_Required):
    """use for full authentication token"""

    def __init__(self, domo_client_id: str, domo_client_secret: str):
        self.domo_client_id = domo_client_id
        self.domo_client_secret = domo_client_secret
        self.domo_instance = ""

    async def generate_auth_header(self, token: str) -> dict:
        if not self.token:
            await self.get_auth_token()

        self.auth_header = {"Authorization": "bearer " + token}
        return self.auth_header

    async def get_auth_token(
        self,
        session: Optional[aiohttp.ClientSession] = None,
    ) -> str:

        res = await get_developer_auth(
            domo_client_id=self.domo_client_id,
            domo_client_secret=self.domo_client_secret,
            session=session,
        )

        if res.status == 401:
            raise InvalidCredentialsError(
                status=res.status,
                message=str(res.response),
                domo_instance=self.domo_instance,
            )

        token = str(res.response.get("access_token"))
        self.token = token
        self.user_id = res.response.get("userId")
        self.domo_instance = res.response.get("domain")
        self.set_manual_login()

        self.auth_header = self.generate_auth_header(token=token)

        if not self.token_name:
            self.token_name = "developer_auth"

        return token

#### Sample implementations of DomoDeveloperAuth


In [ ]:
domo_client_id = "test_client"
domo_client_secret = "test_secret"

try:
    domo_auth = DomoDeveloperAuth(domo_client_id, domo_client_secret)
    await domo_auth.get_auth_token()
except InvalidCredentialsError as e:
    print(e)

Status 401 - Unauthorized


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

Bad pipe message: %s [b'\n\nN\n\x87-\xe0V\xe3\xc5\xfe\xf7\x83=\xec\xcb\xa9\xbd \xde\xea\x9d&\xe1\xac\\\xc9\xe1\xa5f\xfaY\xea,Go"\xed\xf5\x0c\xa2\x0c\xdbZ5\xb4\xc4\x01\xc9\xde/\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'']
Bad pipe message: %s [b'\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 $mG\x96.\xb6\xfc\xa9\x82\x9d\x06yR\xd2Y%\xff\xf4\xe6\x04\xd1\xfd']
Bad pipe message: %s [b'\xaf!p\x1bE\xec\xcc4Z\xc2\x8d\x88\xa0\x17\xd3\x15\x16! \x1f\xa6X@\x1b^\xba\x97\xcd7k\xe0U\xbd\xe2\xacO\xb3L\xa5\x1b\x10\xfd\xb2\xb9\x95\x0c\x85RC"F\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x0